<a href="https://colab.research.google.com/github/orbenishay/python-project/blob/master/final_ver6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Backround


---

We all have typing mitsakes, as we type quickly and we don't want to spend our time veryfing our text. This mini-project tries to assist in detecting them. The idea is to have a script that receives a text input or and returns the mistakes found, as well as an insightful statistical evaluation for the user.
This utility can also be used by children learning english before submitting their work, to verifiy that there are no spell mistakes.

# Project Outline

---





*   Import modules and initiations
*   Creating a class with proper attributes and methods
*   Get user's preferences
*   Implementing the interface
*   Program execution and class instantiation








# Import sys, re modules

In [1]:
# imports and initializations
import sys
import re
from google.colab import files, drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Class SpellChecker definitions & Methods

Init fuctions 

In [0]:
class SpellChecker:
  def __init__(self, input_type="file", typo_count=20, name_ignore=True, check_english_level=True):
      self.input_type = input_type
      self.typo_count = typo_count
      self.name_ignore = name_ignore
      self.check_english_level = check_english_level
      self.text1 = ""
      print(f"Your preferences are: Input Type: {input_type}, Number of errors: {typo_count}, Ignore names: {name_input}")

  def load_text(self):
    if self.input_type == 'text':
      input2 = input("Please enter your text:")
      text1 = input2.split("/n") #was by period
      # print(text1)
    else: # elif input_type.lower() == 'file':
      uploaded = files.upload()
      fname = list(uploaded.keys())[0]
      with open(fname) as f1:
        text1 = []
        for line in f1:
          text1.append(line)
    return text1

 # English-level checker   
  def english_level(self,number_word,number_difficoult_words):
      if number_word>0:
        precent_difficoult_words=number_difficoult_words/number_word
        if precent_difficoult_word>0.1:
          self.englisg_level="is excelent"
        elif precent_difficoult_word>0.01:
          self.english_level="is fine"
        else:
          self.english_level="can be better"
      else:
        self.english_level="can not be evaluated"


# Parsing the text (data cleanning)

  def parse_rows(self, text):
  
    word_dict = {}
    for row_num, sentence in enumerate(text):
      words_list = sentence.split()
      for word_num, word in enumerate(words_list):
        dispo = True
        match_word = re.search('^[a-z]+$', word)
        match_cap = re.match('[A-Z]', word)
        #  need to allow '.' at the end of word as exception
        # need to ignore comma at the end of a word

        if len(word)<=2: # can usae 'or' here for better performance
          dispo = False
        elif bool(match_word)==False:
          dispo = False
        elif self.name_ignore == True and word_num>0 and bool(match_cap)==True:
            dispo = False
            
        word_and_dispo = (word, dispo)
        word_dict[(row_num, word_num)] = word_and_dispo

    return word_dict

 #Get mistakes resaults
  def get_results(self, mistakes):
    for (line,index) in mistakes.keys():
      print(f"In line {str(line)}: the word {mistakes[(line,index)]} was mispelled")


  

In [0]:
#This function checkes if word is in dictionery 
def wordinenglish(word,dic):
  #input: word - a string in English lowcase, dic - dictionery format {"words":[]}, 
  # return boolian if word in dic
  if word in dic:
    return True
  else:
    return False

In [0]:
#Check Spelling- core function
def speller(allwordes,dic,dicall={}):
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  #This function cheks spelling 
  #receives inputs:
  #allwords - All the words to be check, format = {(line,index):["word",True/Falls]} the bulian check or not check spelling for the word
  #dic - primery dictionery, format {"word":[""]} recomended to use friquent words in English to make an effishient allgoritem
  #dicall - secondery dictionery, format {"word":[""]} dictionery with more word in Eglish then dic, diffult value is emty and not be used
  #returns outputs:
  #mistakes - words checked and not at two dictionery, format {(line,index):["word"]}
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  lincounter=0
  wordcounter=0
  mistakecounter1=0
  mistakecounter2=0
  mistakes={}
  for (line,index) in allwordes.keys():
    if mistakecounter2 >= int(typo_count):
      break
    word=allwordes[(line,index)]
    if word[1]: # (if word[1]=True)
        if not wordinenglish(word[0],dic):
          mistakecounter1+=1
        
          if not wordinenglish(word[0],dicall):
            mistakecounter2+=1
            mistakes[(line,index)]=word[0] 

  return mistakes, mistakecounter1, mistakecounter2

# Get user preferences

In [13]:
input_type=""
while  input_type.lower() not in ['t','f']:
  input_type = input("Hello. To enter text please enter \'t\'. To upload a text file please enter \'f\': ")
if input_type.lower() == 't':
  input_type="text"
else:
  input_type="file"


typo_count = "0"
typo_count = input("please enter the amount of errors you would like to find in your text:")
while typo_count.isdigit() == False and typo_count !=0:
  typo_count = input("please enter the amount of errors you would like to find in your text:")

name_input = ""
name_ignore = True
while name_input.lower() not in ['y', 'n']:
  name_input = input("would you like to ignore names in your text? (Y/N)").upper()
  if name_input == 'Y':
    name_ignor = True
  elif name_input == 'N':
    name_ignor = False
check_english_level_input = ""
check_english_level = True
while check_english_level_input.lower() not in ['y', 'n']:
  check_english_level_input = input("would you like to evaluate text English level? (Y/N)").upper()
  if check_english_level_input == 'Y':
    check_english_level = True
  elif check_english_level_input == 'N':
    check_english_level = False


sp =SpellChecker(input_type=input_type,typo_count=typo_count,name_ignore=name_ignor,check_english_level=check_english_level)
if sp.check_english_level:
  english_level_w=" will"
else:
  english_level_w=" will not"

if sp.name_ignore:
  name_ignor_w=" will not"
else:
  name_ignor_w=" will"
print("\nYour preferences are:\n")
print(f"The input is a {sp.input_type}")
print(f"The speller stopes spelling check after {sp.typo_count} speling errores")
print(f"The speller{name_ignor_w} check names")
print(f"The speller{english_level_w} evaluate the text English level")



Hello. To enter text please enter 't'. To upload a text file please enter 'f': f
please enter the amount of errors you would like to find in your text:3
would you like to ignore names in your text? (Y/N)y
would you like to evaluate text English level? (Y/N)y
Your preferences are: Input Type: file, Number of errors: 3, Ignore names: Y

Your preferences are:

The input is a file
The speller stopes spelling check after 3 speling errores
The speller will not check names
The speller will evaluate the text English level


Upload a file or text

In [14]:
text1 = sp.load_text()


Saving task1.txt to task1.txt


Parsing the tex

In [15]:
parsed_text = sp.parse_rows(text1)
print(parsed_text)
print(text1)

{(0, 0): ('The', False), (0, 1): ('following', True), (0, 2): ('are', True), (0, 3): ('the', True), (0, 4): ('graphical', True), (0, 5): ('(non-control)', False), (0, 6): ('characters', True), (0, 7): ('defined', True), (0, 8): ('by', False), (1, 0): ('ISO', False), (1, 1): ('8859-1', False), (1, 2): ('(1987).', False), (1, 3): ('Descriptions', False), (1, 4): ('in', False), (1, 5): ('words', True), (1, 6): ("aren't", False), (1, 7): ('all', True), (1, 8): ('that', True), (1, 9): ('helpful,', False), (2, 0): ('but', True), (2, 1): ("they're", False), (2, 2): ('the', True), (2, 3): ('best', True), (2, 4): ('we', False), (2, 5): ('can', True), (2, 6): ('do', False), (2, 7): ('in', False), (2, 8): ('text.', False), (2, 9): ('A', False), (2, 10): ('graphics', True), (2, 11): ('file', True), (2, 12): ('illustrating', True), (3, 0): ('the', True), (3, 1): ('character', True), (3, 2): ('set', True), (3, 3): ('should', True), (3, 4): ('be', False), (3, 5): ('available', True), (3, 6): ('from',

In [0]:
fname = "/content/drive/My Drive/5000words.txt"
dic={}
with open(fname) as f2:
    for line in f2:
        word=line.split()
        if not(word[0]=="Rank"): #remooving header from the words file
               dic[word[1]]=[word[0],word[2],word[3]]


# print(dic)

fname = "/content/drive/My Drive/allwords.txt"
dicall={}
with open(fname) as f3:
    for line in f3:
        dicall[line.split()[0]]=""
#print(dicall)

In [41]:
m,m1,m2= speller(allwordes=parsed_text,dic=dic, dicall=dicall)
sp.get_results(m)
print(parsed_text)
print(m)

{(0, 0): ('The', False), (0, 1): ('following', True), (0, 2): ('are', True), (0, 3): ('the', True), (0, 4): ('graphical', True), (0, 5): ('(non-control)', False), (0, 6): ('characters', True), (0, 7): ('defined', True), (0, 8): ('by', False), (1, 0): ('ISO', False), (1, 1): ('8859-1', False), (1, 2): ('(1987).', False), (1, 3): ('Descriptions', False), (1, 4): ('in', False), (1, 5): ('words', True), (1, 6): ("aren't", False), (1, 7): ('all', True), (1, 8): ('that', True), (1, 9): ('helpful,', False), (2, 0): ('but', True), (2, 1): ("they're", False), (2, 2): ('the', True), (2, 3): ('best', True), (2, 4): ('we', False), (2, 5): ('can', True), (2, 6): ('do', False), (2, 7): ('in', False), (2, 8): ('text.', False), (2, 9): ('A', False), (2, 10): ('graphics', True), (2, 11): ('file', True), (2, 12): ('illustrating', True), (3, 0): ('the', True), (3, 1): ('character', True), (3, 2): ('set', True), (3, 3): ('should', True), (3, 4): ('be', False), (3, 5): ('available', True), (3, 6): ('from',

# **Spelling report**

---
calling the speller function and present the mistakes, the rate of errors and indication on English level 


In [0]:
mistakes, mistakecounter1, mistakecounter2=speller(text2,dic,dicall)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print("                     Spelling Report\n")
print("Speller found {} spelling mistakes, {:.1%} of the wordes were misspaled".format(len(mistakes),len(mistakes)/len(text2)))
ifcheck_english_level:
  sp.english_level(len(text2),(mistakecounter1-mistakecounter2))
  print("\nThe English level of the texst {}".format(sp.englisg_level_str))
print("\n          This are the spelling mistakes found in the text:\n")

#print(mistakes)

for (line,index) in mistakes.keys():
  print(f"In line {str(line)}, at index {str(index)}: the word {mistakes[(line,index)]} was mispelled")

print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#results(mistakes)